<a href="https://colab.research.google.com/github/Eboselethefirst/Mini_projects/blob/main/SparkML_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls

sample_data


In [ ]:
!pip install pyspark findspark

In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

In [ ]:
import findspark as fs
fs.init()

from pyspark.sql import SparkSession

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

#for regression metrics or evaluators
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
spark = SparkSession.builder.appName("Regression using SparkML").getOrCreate()

In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/mpg.csv

--2026-01-20 16:07:57--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/mpg.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13891 (14K) [text/csv]
Saving to: ‘mpg.csv’

mpg.csv             100%[===================>]  13.57K  --.-KB/s    in 0s      

2026-01-20 16:07:58 (121 MB/s) - ‘mpg.csv’ saved [13891/13891]



In [ ]:
!ls

mpg.csv  sample_data


In [ ]:
mpg_data = spark.read.csv("mpg.csv",header=True,inferSchema=True)
mpg_data.show(7)

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|15.0|        8|      390.0|       190|  3850|       8.5|  70|American|
|21.0|        6|      199.0|        90|  2648|      15.0|  70|American|
|18.0|        6|      199.0|        97|  2774|      15.5|  70|American|
|16.0|        8|      304.0|       150|  3433|      12.0|  70|American|
|14.0|        8|      455.0|       225|  3086|      10.0|  70|American|
|15.0|        8|      350.0|       165|  3693|      11.5|  70|American|
|18.0|        8|      307.0|       130|  3504|      12.0|  70|American|
+----+---------+-----------+----------+------+----------+----+--------+
only showing top 7 rows


In [ ]:
mpg_data.printSchema()

root
 |-- MPG: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Engine Disp: double (nullable = true)
 |-- Horsepower: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Accelerate: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Origin: string (nullable = true)



In [ ]:
#Preparing the feature columns
assembler = VectorAssembler(inputCols=["Cylinders", "Engine Disp", "Horsepower", "Weight", "Accelerate", "Year"],outputCol="features")
mpg_transfromed_data = assembler.transform(mpg_data)


In [ ]:
#spitting the data
(training_data,test_data)= mpg_transfromed_data.randomSplit([0.7,0.3],seed = 42)

In [ ]:
#Building and training the regression model
lr = LinearRegression(featuresCol="features",labelCol="MPG")
model = lr.fit(training_data)

In [ ]:
#Evaluating the model
predictions = model.transform(test_data)

In [ ]:
evaluator = RegressionEvaluator(labelCol="MPG", predictionCol="prediction",metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("RMSE:",rmse)

RMSE: 3.4531049690792264


In [ ]:
evaluator = RegressionEvaluator(labelCol="MPG",predictionCol="prediction",metricName="mae")
mae = evaluator.evaluate(predictions)
print("Mean Absolute Error = ", mae)

Mean Absolute Error =  2.842391179195022


In [ ]:
evaluator = RegressionEvaluator(labelCol="MPG",predictionCol="prediction",metricName="mse")
mse=evaluator.evaluate(predictions)
print("Mean Squared Error is =",mse)

Mean Squared Error is = 11.923933927479645


In [ ]:
spark.stop()

In [ ]:
spark = SparkSession.builder.appName("Diamond Price Prediction").getOrCreate()

In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/diamonds.csv


--2026-01-20 16:51:46--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/diamonds.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3192561 (3.0M) [text/csv]
Saving to: ‘diamonds.csv’

diamonds.csv        100%[===================>]   3.04M  7.02MB/s    in 0.4s    

2026-01-20 16:51:47 (7.02 MB/s) - ‘diamonds.csv’ saved [3192561/3192561]



In [ ]:
#Another example would be:
df2 = spark.read.csv("diamonds.csv",header=True,inferSchema=True)
df2.show(7)

+---+-----+---------+-----+-------+-----+-----+-----+----+----+----+
|  s|carat|      cut|color|clarity|depth|table|price|   x|   y|   z|
+---+-----+---------+-----+-------+-----+-----+-----+----+----+----+
|  1| 0.23|    Ideal|    E|    SI2| 61.5| 55.0|  326|3.95|3.98|2.43|
|  2| 0.21|  Premium|    E|    SI1| 59.8| 61.0|  326|3.89|3.84|2.31|
|  3| 0.23|     Good|    E|    VS1| 56.9| 65.0|  327|4.05|4.07|2.31|
|  4| 0.29|  Premium|    I|    VS2| 62.4| 58.0|  334| 4.2|4.23|2.63|
|  5| 0.31|     Good|    J|    SI2| 63.3| 58.0|  335|4.34|4.35|2.75|
|  6| 0.24|Very Good|    J|   VVS2| 62.8| 57.0|  336|3.94|3.96|2.48|
|  7| 0.24|Very Good|    I|   VVS1| 62.3| 57.0|  336|3.95|3.98|2.47|
+---+-----+---------+-----+-------+-----+-----+-----+----+----+----+
only showing top 7 rows


In [ ]:
df2.printSchema()

root
 |-- s: integer (nullable = true)
 |-- carat: double (nullable = true)
 |-- cut: string (nullable = true)
 |-- color: string (nullable = true)
 |-- clarity: string (nullable = true)
 |-- depth: double (nullable = true)
 |-- table: double (nullable = true)
 |-- price: integer (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



In [ ]:
#Vector assembler
assembler = VectorAssembler(inputCols=["carat","depth","table"],outputCol="features")
df2_with_features = assembler.transform(df2)

In [ ]:
df2_with_features.show(7)

+---+-----+---------+-----+-------+-----+-----+-----+----+----+----+----------------+
|  s|carat|      cut|color|clarity|depth|table|price|   x|   y|   z|        features|
+---+-----+---------+-----+-------+-----+-----+-----+----+----+----+----------------+
|  1| 0.23|    Ideal|    E|    SI2| 61.5| 55.0|  326|3.95|3.98|2.43|[0.23,61.5,55.0]|
|  2| 0.21|  Premium|    E|    SI1| 59.8| 61.0|  326|3.89|3.84|2.31|[0.21,59.8,61.0]|
|  3| 0.23|     Good|    E|    VS1| 56.9| 65.0|  327|4.05|4.07|2.31|[0.23,56.9,65.0]|
|  4| 0.29|  Premium|    I|    VS2| 62.4| 58.0|  334| 4.2|4.23|2.63|[0.29,62.4,58.0]|
|  5| 0.31|     Good|    J|    SI2| 63.3| 58.0|  335|4.34|4.35|2.75|[0.31,63.3,58.0]|
|  6| 0.24|Very Good|    J|   VVS2| 62.8| 57.0|  336|3.94|3.96|2.48|[0.24,62.8,57.0]|
|  7| 0.24|Very Good|    I|   VVS1| 62.3| 57.0|  336|3.95|3.98|2.47|[0.24,62.3,57.0]|
+---+-----+---------+-----+-------+-----+-----+-----+----+----+----+----------------+
only showing top 7 rows


In [ ]:
#Splitting into Training Data and Test data
(training_data,test_data)= df2_with_features.randomSplit([0.7,0.3], seed=42)

In [ ]:
#Building And Training the regression model
lr = LinearRegression(featuresCol="features",labelCol="price")
model = lr.fit(training_data)

In [ ]:

#Evaluating the model with test_data
predictions = model.transform(test_data)

In [ ]:
evaluator = RegressionEvaluator(labelCol="price",predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print("The R squared error is =", r2)

The R squared error is = 0.8535478116135986


In [ ]:
evaluator =RegressionEvaluator(labelCol="price",predictionCol="prediction",metricName="mae")
mae = evaluator.evaluate(predictions)
print("The mean absolute error is:",mae)

The mean absolute error is: 995.4442050140472


In [ ]:
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction",metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("The root mean squared error = ",rmse)

The root mean squared error =  1525.1184531740482
